Visualizing and Understanding Convolutional Networks
----

arXiv: 1311.2901v3 [cs.CV] 28 Nov 2013

Abstract

Large Convolutional Network models have recently demonstrated impressive classification performance on the ImageNet benchmark (Krizhevsky et al., 2012). However there is no clear understanding of why they perform so well, or how they might be improved. In this paper we address both issues. We introduce a novel visualization technique that gives insight into the function of intermediate feature layers and the operation of the classifier. Used in a diagnostic role, these visualizations allow us to find model architec- tures that outperform Krizhevsky et al. on the ImageNet classification benchmark. We also perform an ablation study to discover the performance contribution from different model layers. We show our ImageNet model generalizes well to other datasets: when the softmax classifier is retrained, it convincingly beats the current state-of-the-art results on Caltech-101 and Caltech-256 datasets.

大型的卷积网络最近在ImageNet上显示很强的分类能力。但是对于为什么它们能够表现如此出色，或者如何改进并不是很清楚。这篇文章解决了这两个问题。我们引进了一种新颖的视觉技术，对中间层的属性和分类器的操作给予一些直观的洞悉。用来作为诊断的方法，我们找到了一种更好的结构比Krizhevskey2012更优秀。我们还进行了一些移除的试验来揭示不同的层对于性能的贡献。我们的模型在别的数据集上泛化很好：对最后的softmax分类器重新训练，我们的模型打败了在Caltech-101和Caltech-256上最前沿的技术。

1 Introduction

Since their introduction by (LeCun et al., 1989) in the early 1990’s, Convolutional Networks (convnets) have demonstrated excellent performance at tasks such as hand-written digit classification and face detec- tion. In the last year, several papers have shown that they can also deliver outstanding performance on more challenging visual classification tasks. (Ciresan et al., 2012) demonstrate state-of-the-art performance on NORB and CIFAR-10 datasets. Most notably, (Krizhevsky et al., 2012) show record beating perfor- mance on the ImageNet 2012 classification benchmark, with their convnet model achieving an error rate of 16.4%, compared to the 2nd place result of 26.1%. Several factors are responsible for this renewed inter-est in convnet models: (i) the availability of much larger training sets, with millions of labeled exam- ples; (ii) powerful GPU implementations, making the training of very large models practical and (iii) bet- ter model regularization strategies, such as Dropout (Hinton et al., 2012).

自从convnets在90年代初被LeCun1989引入，在手写数字分类和人脸识别上表现出了很好的效果。去年，几篇论文也显示了它们在更有挑战视觉分类任务上也可以达到很好的性能。（Ciresan2012）在NORB和CIFAR-10上达到了最高的水平。最有名的，Krizhevsky2012在ImageNet2012分类指标上做到非常好的效果，错误率达到16.4%，比第二名好了26.1%。在这个重新被引起兴趣的convnet模型上有几个因素：(i)大型数据集的出现，包含了百万计的标签样本。(ii)计算能力更强的GPU使得模型可以变得更大。(iii)更好的模型正规化测试，例如dropout（Hinton2012）

Despite this encouraging progress, there is still lit- tle insight into the internal operation and behavior of these complex models, or how they achieve such good performance. From a scientific standpoint, this is deeply unsatisfactory. Without clear understanding of how and why they work, the development of better models is reduced to trial-and-error. In this paper we introduce a visualization technique that reveals the in- put stimuli that excite individual feature maps at any layer in the model. It also allows us to observe the evolution of features during training and to diagnose potential problems with the model. The visualization technique we propose uses a multi-layered Deconvo- lutional Network (deconvnet), as proposed by (Zeiler et al., 2011), to project the feature activations back to the input pixel space. We also perform a sensitivity analysis of the classifier output by occluding portions of the input image, revealing which parts of the scene are important for classification.

尽管这些进展非常鼓舞人心，但是对于内部操作和这种复杂的模型的理解，还要他们如何达到这么好的性能的理解还是很少。从科学的立场来看，这非常低不能接受。如果不能很好地理解它们是如何工作的，要开发出更好的模型只是试错和不断的试验。这篇文章，我们提出一种视觉方法可以揭示输入刺激某层的某些特征图。这种视觉方法使用多层的反卷积网络（deconvnet），由Zeiler2011提出。用来把特征图反向生成输入像素空间。我们还做了敏感分析，来发现输入图片的哪些部分对于分类是重要的。

Using these tools, we start with the architecture of (Krizhevsky et al., 2012) and explore different archi- tectures, discovering ones that outperform their results on ImageNet. We then explore the generalization abil- ity of the model to other datasets, just retraining the softmax classifier on top. As such, this is a form of su- pervised pre-training, which contrasts with the unsu- pervised pre-training methods popularized by (Hinton et al., 2006) and others (Bengio et al., 2007; Vincent et al., 2008). The generalization ability of convnet fea- tures is also explored in concurrent work by (Donahue et al., 2013).

使用这些工具，我们从Krizhevsky2012的结构开始，然后探索了不同的结构，发现了能够在ImageNet做到更好的结构。然后我们把模型泛化到其他数据集上，只是重新训练了最后的softmax分类器。这是一种形式的监督预训练，相对于非监督预训练方法（Hinton2006，Bengio2007，Vincent2008）。convnet的泛化能力也在Donahue2013里被研究。

1.1. Related Work

Visualizing features to gain intuition about the network is common practice, but mostly limited to the 1st layer where projections to pixel space are possible. In higher layers this is not the case, and there are limited methods for interpreting activity. (Erhan et al., 2009) find the optimal stimulus for each unit by perform- ing gradient descent in image space to maximize the unit’s activation. This requires a careful initialization and does not give any information about the unit’s invariances. Motivated by the latter’s short-coming, (Le et al., 2010) (extending an idea by (Berkes & Wiskott, 2006)) show how the Hessian of a given unit may be computed numerically around the optimal response, giving some insight into invariances. The problem is that for higher layers, the invariances are extremely complex so are poorly captured by a simple quadratic approximation. Our approach, by contrast, provides a non-parametric view of invariance, showing which pat- terns from the training set activate the feature map. (Donahue et al., 2013) show visualizations that iden- tify patches within a dataset that are responsible for strong activations at higher layers in the model. Our visualizations differ in that they are not just crops of input images, but rather top-down projections that reveal structures within each patch that stimulate a particular feature map.

通过可视化来得到关于网络的直觉这种方式很常见，但是一般仅限于第一层，第一层投射到像素空间才有可能。更高层就不是这样了，用来解释激励的方法很有限。（Erhan2009）找到了一种光学刺激的方法通过图片空间的梯度下降来最大化每个单元的激励。这需要在初始化的时候非常小心，而且没有给出关于单元不变形的任何信息。Le2010指出了单元的Hessian可以计算光学反应，给出了不变形的一些启示。问题在于对于影藏层，不变性是非常复杂的，通过平方估计很难捕捉。相反的，我们的方法提供一种关于不变形的非参数视角，显示了训练集里什么样的模式导致了特征图上的激励。（Donahue2013）显示可视化xxxx。

2 Approach

We use standard fully supervised convnet models throughout the paper, as defined by (LeCun et al., 1989) and (Krizhevsky et al., 2012). These models map a color 2D input image xi, via a series of layers, to a probability vector yˆ over the C different i classes. Each layer consists of (i) convolution of the previous layer output (or, in the case of the 1st layer, the input image) with a set of learned filters; (ii) passing the responses through a rectified linear function (relu(x) = max(x,0)); (iii) [optionally] max pooling over local neighborhoods and (iv) [optionally] a lo- cal contrast operation that normalizes the responses across feature maps. For more details of these opera- tions, see (Krizhevsky et al., 2012) and (Jarrett et al., 2009). The top few layers of the network are conventional fully-connected networks and the final layer is a softmax classifier. Fig. 3 shows the model used in many of our experiments.

本文中我们一直使用标准的监督卷积网络模型。这种模型将一个彩色二维图片$x_i$，通过一系列层，映射到一个对应`C`个分类的概率向量。由这些层构成(i)对前一层使用一组可学习的滤波器做卷积；(ii)把结果传递给relu函数；(iii)【可选】最大池；(iv)【可选】LRN。参见Krizhevsky2012和Jarrett2009见详细解释。后几层习惯上都是fc层，最后一层是softmax分类器。图3显示我们使用的模型。

We train these models using a large set of N labeled images {x,y}, where label yi is a discrete variable indicating the true class. A cross-entropy loss func- tion, suitable for image classification, is used to com-
pare yˆ and y . The parameters of the network (fil-ters in the convolutional layers, weight matrices in the fully-connected layers and biases) are trained by back- propagating the derivative of the loss with respect to the parameters throughout the network, and updating the parameters via stochastic gradient descent. Full details of training are given in Section 3.

我们使用大量带标记的图片{x, y}来训练模型，$y_i$是一个离散变量用来代表真实的类别。损失函数使用cross-entropy，它很适合分类任务，用来比较$y_i$和$\hat{y_i}$。网络的参数通过损失函数梯度反向传递来进行训练（conv中的滤波器，fc中的权重和偏移），通过随机梯度下降来更新。详细的训练过程见第三章。

2.1. Visualization with a Deconvnet

Understanding the operation of a convnet requires in- terpreting the feature activity in intermediate layers. We present a novel way to map these activities back to the input pixel space, showing what input pattern orig- inally caused a given activation in the feature maps. We perform this mapping with a Deconvolutional Net- work (deconvnet) (Zeiler et al., 2011). A deconvnet can be thought of as a convnet model that uses the same components (filtering, pooling) but in reverse, so instead of mapping pixels to features does the oppo- site. In (Zeiler et al., 2011), deconvnets were proposed as a way of performing unsupervised learning. Here, they are not used in any learning capacity, just as a probe of an already trained convnet.

为了理解convnet的操作，需要能解释中间层的特征激励。我们提出一种新颖的方法把这些激励反向映射回输入的像素空间。显示了什么样的输入模式导致了特殊图上的哪些激励。我们称这种映射为反卷积网络(deconvnet)（Zeiler2011）。deconvnet可以被看成一个使用相同组件的convnet（滤波，池化）但是方向相反，不是把像素变成特征，而是反过来。In（Zeiler2011）deconvnet被当做一种无监督学习方法。这里，它们不用来做任何学习，而是作为已经训练好的convnet的探针。

To examine a convnet, a deconvnet is attached to each of its layers, as illustrated in Fig. 1(top), providing a continuous path back to image pixels. To start, an input image is presented to the convnet and features computed throughout the layers. To examine a given convnet activation, we set all other activations in the layer to zero and pass the feature maps as input to the attached deconvnet layer. Then we successively (i) unpool, (ii) rectify and (iii) filter to reconstruct the activity in the layer beneath that gave rise to the chosen activation. This is then repeated until input pixel space is reached.

为了查看一个convnet，一个deconvnet关联到它的每一层，如图1显示，提供了一个连续的反向路径回到图片像素。开始，输入图片进入convnet，特征被计算出来。为了查看一个给定的convnet的激励，我们把其他的激励设置为0，然后把特征图当成输入传递给deconvnet。然后顺序的进行(i)反池化，(ii)rectify，(iii)滤波器来重建。重复这个过程一直到达输入像素层。

![deconvnet.png](deconvnet.png)

Figure 1. Top: A deconvnet layer (left) attached to a convnet layer (right). The deconvnet will reconstruct an ap- proximate version of the convnet features from the layer beneath. Bottom: An illustration of the unpooling oper- ation in the deconvnet, using switches which record the location of the local max in each pooling region (colored zones) during pooling in the convnet.

Unpooling: In the convnet, the max pooling opera- tion is non-invertible, however we can obtain an ap- proximate inverse by recording the locations of the maxima within each pooling region in a set of switch variables. In the deconvnet, the unpooling operation uses these switches to place the reconstructions from the layer above into appropriate locations, preserving the structure of the stimulus. See Fig. 1(bottom) for an illustration of the procedure.

反池化：在convnet中，最大池的操作是不可逆的。但是我们通过一些开关变量记录每个池区域最大值的位置，来得到近似的反池化。

Rectification: The convnet uses relu non-linearities, which rectify the feature maps thus ensuring the fea- ture maps are always positive. To obtain valid fea- ture reconstructions at each layer (which also should be positive), we pass the reconstructed signal through a relu non-linearity.

整流：convnet使用relu非线性激励，用来改变特征图保证特征始终为正。为了得到合法的特征重建，我们把重建信号传递给relu。

Filtering: The convnet uses learned filters to convolve the feature maps from the previous layer. To invert this, the deconvnet uses transposed versions of the same filters, but applied to the rectified maps, not the output of the layer beneath. In practice this means flipping each filter vertically and horizontally.

滤波：convnet使用滤波器把特征图和前一层卷积到一起。为了反转这个，deconvnet使用同样的滤波器的转置版本，应用到整流后的图上，而不是前一层的输出上。实际上这意味着把滤波器水平和垂直都翻转了。

Projecting down from higher layers uses the switch settings generated by the max pooling in the convnet on the way up. As these switch settings are peculiar to a given input image, the reconstruction obtained from a single activation thus resembles a small piece of the original input image, with structures weighted according to their contribution toward to the feature activation. Since the model is trained discriminatively, they implicitly show which parts of the input image are discriminative. Note that these projections are not samples from the model, since there is no generative process involved.

反池化用到的开关依赖于池化时的值。这些开关是针对于给定的输入图片的。